In [1]:
import dash
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State
import numpy as np

In [2]:
app = Dash(
    external_stylesheets=[dbc.themes.DARKLY], update_title='Loading...'
)

In [3]:
#Side Bar Code
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": "10rem",
    "left": 0,
    "bottom": 0,
    "width": "25rem",
    "padding": "2rem 1rem",
    "background-color": "#222",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sideBar = html.Div([
        html.H3("Filter", className="display-4"),
        html.Hr(),
        html.P(
            "A simple sidebar layout with navigation links", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="sidebar-content", style=CONTENT_STYLE)


In [4]:
#Title Code
TITLE_STYLE = {
    "position": "flex",
    "top": 0,
    "padding": "2rem 2rem",
}

dashboardTitle = html.Div([
                html.H1("Egypt Job Posts 2022", className="twelve columns", style={"text-align": "center"})
    ], style=TITLE_STYLE,  className = 'row', 
)

In [5]:
#cards
CARD_STYLE={
    "width": "18rem"
}

card1 = dbc.Card(
    [
        
        dbc.CardBody(
            html.P("This card 1", className="card-text", style={"text-align": "center"})
        ),
    ],
    style=CARD_STYLE,
)

card2 = dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 2", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

card3 = dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 3", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

card4= dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 4", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

cards = dbc.Row(
    [
        dbc.Col(card1, width="auto"),
        dbc.Col(card2, width="auto"),
        dbc.Col(card3, width="auto"),
        dbc.Col(card4, width="auto"),
    ]
)

In [6]:
app.layout = html.Div([
            dashboardTitle, sideBar, cards
                            
            ])

In [ ]:
app.title = 'Egypt Job Posts 2022'
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1647776574.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash/deps/react@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_3_0m1647776574.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_3m1648451819.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13:54:43] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_0m1647776574.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2022 13